In [1]:
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
import google.generativeai as genai
from decouple import config


c:\Users\yugpa\OneDrive\Documents\AI_Summarization_project\Environment\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['GOOGLE_API_KEY']=config("GOOGLE_API_KEY")


In [3]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Parul_University")
docs = loader.load()
docs


USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Parul_University', 'title': 'Parul University - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nParul University - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory, gove

In [4]:
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate


text_splitter = TokenTextSplitter.from_tiktoken_encoder(
    chunk_size=8000, chunk_overlap=10
)
split_docs = text_splitter.split_documents(docs)
print(f"Number of chunk created after splitting :- {len(split_docs)}")


Number of chunk created after splitting :- 2


In [5]:
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85)

# To extract data from WebBaseLoader
doc_prompt = PromptTemplate.from_template("{page_content}")

# To query Gemini
llm_prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
llm_prompt = PromptTemplate.from_template(llm_prompt_template)


stuff_chain = (
    # Extract data from the documents and add to the key `text`.
    {
        "text": lambda docs: "\n\n".join(
            format_document(doc, doc_prompt) for doc in docs
        )
    }
    | llm_prompt         # Prompt for Gemini
    | llm                # Gemini function
    | StrOutputParser()  # output parser
)
   

In [6]:
stuff_chain.invoke(docs)


'Parul University is a private university located in Vadodara, Gujarat, India. Established in 2009, it offers over 450 diploma, undergraduate, postgraduate, and doctoral programs in various disciplines. The university is recognized by the University Grants Commission (UGC) and has received an A++ accreditation from the National Assessment and Accreditation Council (NAAC). Parul University has a global presence with over 2,000 international students representing 56 different nationalities. It has collaborations with over 50 international universities and offers various exchange programs. The university emphasizes entrepreneurship and innovation and is ranked among the top 50 universities in India for innovation. It also has a strong commitment to community welfare, particularly in healthcare, and operates Parul Sevashram Hospital, a NABH-accredited teaching hospital.'